In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,precision_score
from sklearn.metrics import roc_curve, roc_auc_score,auc
import xgboost as xgb

In [2]:
data_test=pd.read_csv("data\\test.csv",sep=',')
data_train=pd.read_csv("data\\train.csv",sep=',')

In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

# Étape 1 : Préparez les données
X = data_train.drop(columns=['Exited'])  # Toutes les colonnes sauf 'Exited' pour les variables explicatives
y = data_train['Exited']                 # Cible (1 = churn, 0 = non-churn)

# Étape 2 : Divisez data_train en sous-ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Étape 3 : Encodage des variables catégorielles
# Supprimez les colonnes inutiles
if 'Surname' in X_train.columns:
    X_train = X_train.drop(columns=['Surname'])
    X_val = X_val.drop(columns=['Surname'])

# Encodez les variables catégorielles en utilisant One-Hot Encoding
categorical_columns = [col for col in ['Geography', 'Gender'] if col in X_train.columns]
X_train = pd.get_dummies(X_train, columns=categorical_columns, drop_first=True)
X_val = pd.get_dummies(X_val, columns=categorical_columns, drop_first=True)

# Assurez-vous que les colonnes de X_train et X_val sont les mêmes après l'encodage
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

# Étape 4 : Entraînez le modèle XGBoost
model = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
model.fit(X_train, y_train)

# Étape 5 : Prédisez les probabilités pour l'ensemble de validation
y_pred_proba = model.predict_proba(X_val)[:, 1]  # Probabilité pour la classe 1 (churn)
y_pred = model.predict(X_val)                     # Prédictions de classes pour d'autres métriques

# Calcul de l'AUC pour évaluer la capacité du modèle à distinguer entre churn et non-churn
auc_score = roc_auc_score(y_val, y_pred_proba)
print("AUC:", auc_score)

# Calcul de la précision
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

# Matrice de confusion pour mieux comprendre les erreurs de classification
conf_matrix = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:\n", conf_matrix)


c:\Users\lamou\Downloads\M2_MOSEF_S1\Data Mining\Projet_challenge\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:32:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


AUC: 0.9091831995571634
Accuracy: 0.8806666666666667
Confusion Matrix:
 [[3411  192]
 [ 345  552]]


In [4]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [9]:
# Créez sample_submission en copiant l'identifiant 'id' de data_train pour les échantillons de validation
sample_submission = data_train[['id']].iloc[y_val.index].copy().reset_index(drop=True)

# Ajoutez la colonne 'Churn_Probability' de proba_df à sample_submission
sample_submission["Exited"] = y_pred_proba  # Ajout des probabilités de churn

# Affichez les premières lignes pour vérifier
print(sample_submission.head())


      id    Exited
0   4747  0.000768
1  11140  0.218810
2    404  0.085640
3   8707  0.045092
4   5046  0.012833


In [14]:
sample_submission.to_csv("sample_submission.csv",index=False)

In [16]:
sample_submission.shape

(4500, 2)

In [18]:
# Préparation de data_test pour les prédictions
X_test = data_test.drop(columns=['id'])  # Suppression de la colonne 'id' si elle n'est pas une variable explicative

# Suppression de colonnes inutiles dans data_test
if 'Surname' in X_test.columns:
    X_test = X_test.drop(columns=['Surname'])

# Encodage des variables catégorielles dans data_test
X_test = pd.get_dummies(X_test, columns=categorical_columns, drop_first=True)

# Assurez-vous que les colonnes de X_test correspondent à celles de X_train après l'encodage
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Prédisez les probabilités pour data_test
y_test_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilité de churn pour chaque client dans data_test

# Créez sample_submission avec 'id' et 'Churn_Probability'
sample_submission_test = data_test[['id']].copy()  # Copie uniquement la colonne 'id' de data_test
sample_submission_test["Exited"] = y_test_pred_proba  # Ajout des probabilités de churn

# Affichez les premières lignes de sample_submission pour vérification
print(sample_submission_test.head())

# Optionnel : Exportez sample_submission en CSV
sample_submission_test.to_csv("sample_submission_test.csv", index=False)


      id    Exited
0  15000  0.060819
1  15001  0.000460
2  15002  0.001177
3  15003  0.919986
4  15004  0.014336


In [19]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

param_grid = {
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'n_estimators': [100, 200, 500],
    'colsample_bytree': [0.3, 0.7, 1.0],
    'subsample': [0.5, 0.7, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 5, 10]
}

model = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=20, scoring='roc_auc', cv=3, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
y_pred_proba_best = best_model.predict_proba(X_val)[:, 1]
auc_best = roc_auc_score(y_val, y_pred_proba_best)
print("Best AUC after hyperparameter tuning:", auc_best)


c:\Users\lamou\Downloads\M2_MOSEF_S1\Data Mining\Projet_challenge\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:50:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best AUC after hyperparameter tuning: 0.9248192466887033


In [20]:
from sklearn.model_selection import cross_val_score

model = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
auc_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
print("Cross-validated AUC scores:", auc_scores)
print("Average AUC:", auc_scores.mean())


c:\Users\lamou\Downloads\M2_MOSEF_S1\Data Mining\Projet_challenge\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:51:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\lamou\Downloads\M2_MOSEF_S1\Data Mining\Projet_challenge\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:51:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\lamou\Downloads\M2_MOSEF_S1\Data Mining\Projet_challenge\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:51:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learn

Cross-validated AUC scores: [0.92274519 0.92698229 0.91865707 0.92311515 0.93091537]
Average AUC: 0.9244830133817155


In [26]:

y_pred_proba_best = best_model.predict_proba(X_test)[:, 1]

# Créez sample_submission avec 'id' et 'Churn_Probability'
sample_submission_test_best = data_test[['id']].copy()  # Copie uniquement la colonne 'id' de data_test
sample_submission_test_best["Exited"] = y_pred_proba_best  # Ajout des probabilités de churn

# Affichez les premières lignes de sample_submission pour vérification
print(sample_submission_test_best.head())

# Optionnel : Exportez sample_submission en CSV
sample_submission_test_best.to_csv("sample_submission_test_best.csv", index=False)


      id    Exited
0  15000  0.166050
1  15001  0.058086
2  15002  0.059625
3  15003  0.586088
4  15004  0.081252
